<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/3_6_Pipeline_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Based on: https://www.askpython.com/python/examples/pipelining-in-python
# https://sefidian.com/2022/08/30/a-tutorial-on-scikit-learn-pipeline-columntransformer-and-featureunion/

# Importar librerias
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA


# Seleccion de features y modelos
from pandas import read_csv

from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Pipeline y Feature Union
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

In [4]:

# Carga de datos

url_data = "https://raw.githubusercontent.com/armandoordonez/eda_couse/refs/heads/main/data/pima-indians-diabetes.data.csv"

varnames = ['embarazo', 'var_plas', 'presion', 'piel', 'prueba', 'masa', 'var_pedi', 'edad', 'diabetes']

vardataframe = read_csv(url_data, names=varnames)

vardataframe.head()



,embarazo,var_plas,presion,piel,prueba,masa,var_pedi,edad,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# partimos los datos en X y Y

vararray = vardataframe.values

Xtrain = vararray[:,0:8]
Ytrain = vararray[:,8]

In [10]:
# Feature union

urlfeatures = []

urlfeatures.append(('pca', PCA(n_components=3)))

urlfeatures.append(('select_best', SelectKBest(k=6)))

feature_union = FeatureUnion(urlfeatures)

# La diferentcia entre pipelines y feature union es que en pipelines se ejecutan en serie y en feature union en paralelo


print(feature_union)

FeatureUnion(transformer_list=[('pca', PCA(n_components=3)),
                               ('select_best', SelectKBest(k=6))])


In [11]:
# Crear el Pipeline incluyendo la regresión logística

estimators = []
estimators.append(('feature_union', feature_union))
estimators.append(('logistic', LogisticRegression(max_iter=1000)))

model = Pipeline(estimators)

model

Pipeline(steps=[('feature_union',
                 FeatureUnion(transformer_list=[('pca', PCA(n_components=3)),
                                                ('select_best',
                                                 SelectKBest(k=6))])),
                ('logistic', LogisticRegression(max_iter=1000))])

In [12]:

# Prueba del pipeline

seed = 7

varkfold = KFold(n_splits=10)

dataresults = cross_val_score(model, Xtrain, Ytrain, cv=varkfold)

print(dataresults.mean())

0.7760423786739576
